In [ ]:
from datetime import datetime
from itertools import zip_longest
import tweepy, json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gp
import contextily as ctx
from shapely.geometry import Point

%matplotlib inline

pd.set_option('max_colwidth',280)

In [ ]:
def round_labels(ax):
    """
    Rounds the labels in the legends in a chloropleth so they are ints, not floats
    """
    labels = ax.get_legend().get_texts()
    for lbl in labels:
        label_text = lbl.get_text()
        lower = label_text.split()[0]
        upper = label_text.split()[2]
        new_text = f'{float(lower):,.0f} - {float(upper):,.0f}'
        lbl.set_text(new_text)

## Police Precincts

In [ ]:
# load precint geo dataframe (from: https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz)
precincts = gp.read_file('police_precincts.geojson')

In [ ]:
precincts = precincts.to_crs(epsg=3857)

In [ ]:
# make columns numeric
cols = ['precinct', 'shape_area', 'shape_leng']
precincts[cols] = precincts[cols].apply(pd.to_numeric)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
precincts.plot(ax=ax, alpha=0.5, edgecolor='dimgray', color='#005DB9')

for i, point in precincts.set_index('precinct').buffer(0).representative_point().iteritems():
    ax.annotate(s=i, xy=[point.x - 700, point.y - 400], color='k', fontsize=10)

plt.title('NYPD Precincts', fontsize=18, fontweight='bold', loc='left')    

ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)
plt.savefig('precincts.png', bbox_inches='tight');

## Bike lanes

In [ ]:
# load bike lanes geo dataframe (can be found here: https://data.cityofnewyork.us/Transportation/Bicycle-Routes/7vsa-caz7)
bike_lanes = gp.read_file('bicycle_routes.geojson')
bike_lanes = bike_lanes.to_crs(epsg=3857)

In [ ]:
# fix data types
bike_lanes['lanecount'] = pd.to_numeric(bike_lanes.lanecount)
bike_lanes['instdate'] = pd.to_datetime(bike_lanes.instdate, format='%Y-%m-%d')
bike_lanes['instdate'] = bike_lanes.instdate.replace(datetime(1900, 1, 1), np.nan)
bike_lanes['moddate'] = pd.to_datetime(bike_lanes.moddate, format='%Y-%m-%d', errors='coerce')

In [ ]:
# drop off-road bike lanes
idxs = bike_lanes[bike_lanes.onoffst == 'OFF'].index
bike_lanes_clean = bike_lanes.drop(idxs)

In [ ]:
# drop 'bike-friendly parking' lanes
idxs = bike_lanes_clean[(bike_lanes_clean.tf_facilit == 'Bike-Friendly Parking') | (bike_lanes_clean.ft_facilit == 'Bike-Friendly Parking')].index
bike_lanes_clean = bike_lanes_clean.drop(idxs)

In [ ]:
# drop duplicates
bike_lanes_clean = bike_lanes_clean.drop_duplicates()

In [ ]:
def calculate_length(geom, bike_lanes_clean):
    """Takes one geometry and one geo dataframe and returns length of the intersection of the geometry
    with all points in the geo dataframe"""
    length = bike_lanes_clean.intersection(geom).length.sum()
    return length

In [ ]:
# apply calculate_length to precincts to calculate length of bike lanes in each precinct (in m)
precincts['bike_lane_length'] = precincts.buffer(0).geometry.apply(calculate_length, bike_lanes_clean=bike_lanes_clean)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
precincts.plot(ax=ax, alpha=0.5, edgecolor='dimgray', color='#005DB9')
bike_lanes_clean.plot(ax=ax, color='green')

plt.title('NYC bike lanes by precinct', fontsize=18, fontweight='bold', loc='left')  

ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)

In [ ]:
# modify cmap so it doesn't start at white
cmap = mpl.cm.Blues(np.linspace(0,1,20))
cmap = mpl.colors.ListedColormap(cmap[3:,:-1])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
precincts.plot(ax=ax, column='bike_lane_length', alpha=0.9, edgecolor='k', legend=True,
              scheme='equal_interval', k=5, cmap=cmap,
              legend_kwds={'loc': 'lower right', 'facecolor':'linen', 'title':'Bike lane length (m)'})

round_labels(ax)

ax.set_axis_off()
plt.title('On-road NYC bike lane length by precinct', fontsize=18, fontweight='bold', loc='left')

top_5 = precincts.sort_values(by=['bike_lane_length'], ascending=False).head()

for i, point in top_5.set_index('precinct').representative_point().iteritems():
    ax.annotate(s=i, xy=[point.x - 700, point.y - 200], color='w', fontsize=10)

ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)
plt.savefig('lane_by_precinct.png', bbox_inches='tight');

## Traffic violations

In [ ]:
# load selected columns of violations (from: https://data.cityofnewyork.us/City-Government/Open-Parking-and-Camera-Violations/nc67-uf89)
cols = ['Summons Number', 'Issue Date', 'Violation', 'Precinct', 'County']
violations = pd.read_csv('Open_Parking_and_Camera_Violations.csv', usecols=cols)

In [ ]:
# rename columns
violations.columns = violations.columns.str.replace(' ', '_').str.lower()

In [ ]:
# convert date to datetime
violations['issue_date'] = pd.to_datetime(violations['issue_date'], format='%m/%d/%Y', errors='coerce')

In [ ]:
# drop rows with missing date
violations = violations.dropna(axis=0, subset=['issue_date'])

In [ ]:
# make violation not upper
violations['violation'] = violations.violation.str.capitalize()

In [ ]:
# set date as index
violations.set_index('issue_date', inplace=True)

In [ ]:
bike_violations = violations[violations.violation == 'Bike lane'].copy()
bike_violations = bike_violations[bike_violations.index < datetime(2019, 12, 1)]
bike_viol_by_month = bike_violations.resample('M').size()


fig, ax = plt.subplots(figsize=(20, 10))
bike_viol_by_month.loc['2016':'2019'].plot(fontsize=14, linewidth=3)
plt.annotate('Citywide Bicycle Safe Passage Initiative', fontsize=12,
             xy=('2019-07', bike_viol_by_month.loc['2019-07']),
             xytext=(-260, -50), textcoords='offset points',
             arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=90,angleB=0,rad=10"))

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel('Violation date', fontsize=14)
plt.ylabel('Number of summons', fontsize=14)
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.title('Bike lane summons in NYC', loc='left', fontsize=18, fontweight='bold')
plt.savefig('timeline.png', bbox_inches = "tight");

In [ ]:
# extract 2019 violations only
violations_2019 = violations.loc['2019']

In [ ]:
# drop rows that happened after Nov 2019
violations_2019 = violations_2019[violations_2019.index < datetime(2019, 12, 1)]

In [ ]:
# plot top 25 violations in 2019
bike_prop = 100 * len(violations_2019[violations_2019.violation == 'Bike lane']) / violations_2019.shape[0]

ax = violations_2019.violation.value_counts()[:25].iloc[::-1].plot(kind='barh', figsize=(20, 12), fontsize=14,
                                                                   width=0.8, color='tab:gray')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_title('Top 25 Traffic Violations in NYC in 2019', loc='left', fontsize=18, fontweight='bold')

ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

for patch in ax.patches:
    ax.text(patch.get_width()+15000, patch.get_y()+0.2, '{:,}'.format(patch.get_width()), fontsize=14)

ax.text(600000, 4, f'Bike lane violations represented {round(bike_prop, 2)}% of NYC traffic violations in 2019',
        fontsize=16, fontweight='bold', color='tab:blue')

ax.patches[4].set_color('tab:blue')
ax.get_xaxis().set_visible(False)
plt.savefig('top_violations.png', bbox_inches = "tight");

In [ ]:
# select 2019 bike lane violations only
bike_violations_2019 = violations_2019[violations_2019.violation == 'Bike lane'].copy()

In [ ]:
# make precinct an integer
bike_violations_2019['precinct'] = pd.to_numeric(bike_violations_2019['precinct'], downcast='integer')

In [ ]:
# drop rows with non-existent (i.e. missing) precincts
bike_violations_2019 = bike_violations_2019[bike_violations_2019.precinct.isin(precincts.precinct)]

In [ ]:
# calculate number of bike violations by precinct
bike_viol_19_by_precinct = bike_violations_2019.groupby('precinct').size().to_frame(name='n_bike_violations').reset_index()

In [ ]:
# merge precincts with number of bike violations in 2019 & fill nan with 0
precincts_merged = pd.merge(precincts, bike_viol_19_by_precinct, how='left')
precincts_merged['n_bike_violations'].fillna(0, downcast='int', inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
precincts_merged.plot(column='n_bike_violations', ax=ax, scheme='equal_interval', k=5, cmap='Blues', 
                      legend=True, alpha=0.9, edgecolor='k', 
                      legend_kwds={'loc': 'lower right', 'facecolor':'linen', 'title':'Bike lane violations'})

round_labels(ax)

ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)
plt.title('Bike lane violations by police precinct (2019)', loc='left', fontsize=18, fontweight='bold');

In [ ]:
# add column to precincts calculating violations per km of bike lane in each precinct
precincts_merged['violations_per_km'] = precincts_merged.n_bike_violations / (precincts_merged.bike_lane_length / 1000)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
precincts_merged.plot(column='violations_per_km', ax=ax, scheme='equal_interval', k=5, cmap='Blues', legend=True, 
                      alpha=0.9, edgecolor='k', 
                      legend_kwds={'loc': 'lower right', 'facecolor':'linen', 'title':'Violations per km of bike lane'})

top_5 = precincts_merged.sort_values(by=['violations_per_km'], ascending=False).head()

for i, point in top_5.set_index('precinct').representative_point().iteritems():
    ax.annotate(s=i, xy=[point.x - 700, point.y - 200], color='w', fontsize=10)

round_labels(ax)
ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)
plt.title('Bike lane violations per km of bike lane by police precinct (2019)', loc='left', fontsize=18, fontweight='bold')
plt.savefig('vio_per_km.png', bbox_inches='tight');

In [ ]:
bike_viol_H1_2019 = violations[violations.violation == 'Bike lane'].loc['2019-01':'2019-06']
bike_viol_H1_2018 = violations[violations.violation == 'Bike lane'].loc['2018-01':'2018-06']

print(f'Summons 1H 2018: {bike_viol_H1_2018.shape[0]}')
print(f'Summons 1H 2019: {bike_viol_H1_2019.shape[0]}')

In [ ]:
print('Number of summonses in June-July 2019')
print('June 2019:', bike_violations_2019 .loc['2019-06'].shape[0]) 
print('July 2019:', bike_violations_2019 .loc['2019-07'].shape[0])

## 311 Data

In [ ]:
# load 311 data (from: https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)
cols = ['Unique Key', 'Created Date', 'Complaint Type', 'Descriptor', 'Latitude', 'Longitude']
reports_311 = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', usecols=cols)

In [ ]:
# rename columns
reports_311.columns = reports_311.columns.str.replace(' ', '_').str.lower()

In [ ]:
# convert date to datetime
reports_311['created_date'] = pd.to_datetime(reports_311['created_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [ ]:
# set date as index
reports_311.set_index('created_date', inplace=True)

In [ ]:
# get data for 2019 only
reports_311_2019 = reports_311.loc['2019'].copy()

In [ ]:
# get bike reports only
bike_reports_2019 = reports_311_2019[reports_311_2019.descriptor == 'Blocked Bike Lane'].copy()

In [ ]:
# drop reports with missing coordinates
bike_reports_2019 = bike_reports_2019[bike_reports_2019.latitude.notna()]

In [ ]:
# create a geometry column 
geometry = [Point(xy) for xy in zip(bike_reports_2019.longitude, bike_reports_2019.latitude)]

# Coordinate reference system
crs = {'init': 'epsg:4326'}

# make GeoDataFrame
bike_reports_2019 = gp.GeoDataFrame(bike_reports_2019, crs=crs, geometry=geometry)

In [ ]:
bike_reports_2019 = bike_reports_2019.to_crs(epsg=3857)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
bike_reports_2019.plot(ax=ax, markersize=2)
ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)

In [ ]:
# join geo dataframes to get which reports happened in which precinct
bike_reports_joined = gp.sjoin(bike_reports_2019, precincts_merged, how='inner', op='within')

In [ ]:
# get number of bike reports by precinct
bike_reports_by_precinct = bike_reports_joined.groupby('precinct').size().to_frame(name='n_311_reports').reset_index()

In [ ]:
# merge bike reports by precinct onto precincts DF
precincts_merged = pd.merge(precincts_merged, bike_reports_by_precinct, how='left')
precincts_merged['n_311_reports'].fillna(0, downcast='int', inplace=True)

## Twitter data

311 data is incomplete. I know that some bike lane reports are done to 311 not under the 'Blocked Bike Lane' complaint type, but under the 'For Hire Vehicle Complaint'. Specifically, I know a lot of these reports are done using a popular App called [Reported](https://reportedly.weebly.com/). Even though all of these reports are included in the 311 database, it is impossible to know which of the 'For Hire Vehicle Complaint' reports are due to bike lane obstruction vs any other issue with for-hire vehicles. I therefore decided to supplement the 311 'Blocked Bike Lane' reports with some reports done with and tweeted by the Reported app.

In [ ]:
# open file with scraped tweet ids (run scrape.py first)
with open('all_ids.json', 'r') as json_file:
    tweet_ids = json.load(json_file)

In [ ]:
# authorize and initialize API
access_token = ''
access_token_secret = ''
consumer_key = ''
consumer_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
# iterate over 100-id chunks and get date and text for each tweet id
ids = []
tweet_dates = []
tweet_texts = []
for chunk in grouper(tweet_ids, 100):
    statuses = api.statuses_lookup(chunk, tweet_mode='extended')
    ids.extend([status.id for status in statuses])
    tweet_dates.extend([status.created_at for status in statuses])
    tweet_texts.extend([status.full_text for status in statuses])

In [ ]:
# combine id, date and text into a DataFrame
tweets_dict = {'id':ids, 'date':tweet_dates, 'text':tweet_texts}
tweets = pd.DataFrame(tweets_dict)

In [ ]:
# select tweets that relate to bike lane only
bike_tweets = tweets[tweets.text.str.contains('BlockedBikeNYC')].copy()

In [ ]:
# select tweets that deal with taxis only
bike_tweets = bike_tweets[bike_tweets.text.str.contains('#nyctaxi')].copy()

In [ ]:
# extract precint number from tweet text`
bike_tweets['precinct'] = bike_tweets.text.str.extract(r'#NYPD(\d+|MTS)')

In [ ]:
# drop rows with no precinct
bike_tweets.dropna(inplace=True)

In [ ]:
# replace MTS (Midtown South) for precinct number and make it an integer
bike_tweets['precinct'] = bike_tweets.precinct.str.replace('MTS', '14', regex=False)
bike_tweets['precinct'] = pd.to_numeric(bike_tweets.precinct)

In [ ]:
# get number of reports by precinct
bike_tweets_by_precinct = bike_tweets.groupby('precinct').size().to_frame(name='n_twitter_reports').reset_index()

In [ ]:
# merge precincts df with twitter reports
precincts_merged = pd.merge(precincts_merged, bike_tweets_by_precinct, how='left')
precincts_merged['n_twitter_reports'].fillna(0, downcast='int', inplace=True)

In [ ]:
# new column with total reports (311 + twitter)
precincts_merged['total_reports'] = precincts_merged.n_311_reports + precincts_merged.n_twitter_reports

In [ ]:
# calculate new column with reports per km of bike lane
precincts_merged['reports_per_km'] = precincts_merged.total_reports / (precincts_merged.bike_lane_length / 1000)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
precincts_merged.plot(column='reports_per_km', ax=ax, scheme='quantiles', k=5, cmap='Blues', legend=True, alpha=0.9, edgecolor='k',
                      legend_kwds={'loc': 'lower right', 'facecolor':'linen', 'title':'311 reports per km of bike lane'})

limit = precincts_merged.reports_per_km.quantile(q=0.8)
top_quantile = precincts_merged[precincts_merged.reports_per_km > limit]

for i, point in top_quantile.set_index('precinct').buffer(0).representative_point().iteritems():
    ax.annotate(s=i, xy=[point.x - 700, point.y - 200], color='w', fontsize=10)

labels = ax.get_legend().get_texts()
new_labels = ['Bottom quintile', '2nd quintile', '3rd quintile', '4th quintile', 'Top quintile']
for lbl, new_lbl in zip(labels, new_labels):
    lbl.set_text(new_lbl)

ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.VoyagerNoLabels)
plt.title('Bike lane reports per km of bike lane by police precinct (2019)', loc='left', fontsize=18, fontweight='bold')
plt.savefig('reports.png', bbox_inches='tight');